In [1]:
import numpy as np
import pandas as pd
from functools import reduce

In [2]:
def create_comparison_df():
    xlsx_path = 'evaulation/30 sample penyakit - hasil prompt LLM.xlsx'
    sheets = {
        'Dr': 'Dr Diagnose',
        'Claude': 'Claude 3.5 Haiku',
        'Qwen': 'Qwen 2.5 72B',
        'GPT': 'GPT-4o mini',
        'Deepseek RAG': 'Deepseek-V3-RAG',
        'Deepseek non RAG': 'Deepseek-V3-(nonRAG)'
    }
    
    processed_df_list = []

    for name, sheet_name in sheets.items():
        df = pd.read_excel(xlsx_path, sheet_name=sheet_name)
        df['No'] = df['No'].ffill()
        df['Question'] = df['Question'].ffill()
        
        aggregation_logic = {
            'Answer': lambda ans: ', '.join(ans.dropna().astype(str)),
        }
        if 'Full Answer' in df.columns:
            aggregation_logic['Full Answer'] = 'first'
            

        grouped_df = df.groupby(['No', 'Question'], as_index=False).agg(aggregation_logic)
        rename_map = {'Answer': f'{name} Answer'}

        if 'Full Answer' in grouped_df.columns:
            rename_map['Full Answer'] = f'{name} Full Answer'
        grouped_df = grouped_df.rename(columns=rename_map)
        
        processed_df_list.append(grouped_df)

    final_df = reduce(lambda left, right: pd.merge(left, right, on=['No', 'Question'], how='left'), processed_df_list)
    
    id_cols = ['No', 'Question']
    answer_cols = [f'{name} Answer' for name in sheets]
    full_answer_cols = [f'{name} Full Answer' for name in sheets if name != 'Dr'] 

    final_column_order = id_cols + answer_cols + full_answer_cols
    final_df = final_df[final_column_order]
    return final_df

In [3]:
comparison_df = create_comparison_df()
comparison_df

,No,Question,Dr Answer,Claude Answer,Qwen Answer,GPT Answer,Deepseek RAG Answer,Deepseek non RAG Answer,Claude Full Answer,Qwen Full Answer,GPT Full Answer,Deepseek RAG Full Answer,Deepseek non RAG Full Answer
0,1.0,Pasien mengalami demam pada waktu malam. Sebel...,"Demam Tifoid 70%, Demam Berdarah Dengue 30%","Demam Tifoid, Leptospirosis, Demam Berdarah De...","Demam Biasa, Dispepsia, Asma, Infeksi Saluran ...","Demam Biasa, Demam Tifoid, Demam Berdarah Deng...","Demam Tifoid (Typhoid Fever), Malaria, Leptosp...","Malaria, Infeksi Saluran Kemih (ISK) atau Piel...","Berdasarkan gejala yang disampaikan, saya akan...","Berdasarkan gejala yang disampaikan, yaitu dem...","Berdasarkan gejala yang disampaikan, yaitu dem...","Berdasarkan gejala yang dialami pasien, yaitu ...",Berdasarkan gejala yang Anda sebutkan—demam pa...
1,2.0,Pasien mengalami buang air besar cair lebih da...,"Gastroenteritis (GE) Akut 70%, Intoleransi Lak...","Gastroenteritis (GE) Akut, Disentri, Infeksi E...","Diare Akut, Gastroenteritis, Dispepsia, Infeks...","Gastroenteritis Akut, Infeksi Escherichia coli...","Gastroenteritis Akut, Keracunan Makanan, Infek...","Gastroenteritis Akut, Keracunan Makanan, Koler...","Berdasarkan informasi klinis yang diberikan, s...","Berdasarkan gejala yang disampaikan, yaitu bua...","Berdasarkan informasi yang diberikan, pasien m...","Berdasarkan informasi yang diberikan, pasien m...",Berdasarkan gejala yang Anda deskripsikan—diar...
2,3.0,Pasien datang dengan keluhan demam selama 6 ha...,DBD 80%,"Demam Tifoid, Malaria, DBD, Leptospirosis, Inf...","Demam Biasa, Dispepsia, Asma, Demam Dengue, Lu...","Demam Berdarah Dengue, Demam Tifoid, Malaria, ...","Demam Berdarah Dengue (DBD), Demam Tifoid, Lep...","Demam Berdarah Dengue (DBD) / Dengue Fever, Ch...","Berdasarkan informasi yang diberikan, saya aka...","Berdasarkan gejala yang disampaikan, yaitu dem...",Berdasarkan keluhan yang disampaikan oleh pasi...,Berdasarkan keluhan pasien yang meliputi demam...,Berdasarkan gejala yang Anda sebutkan—demam ≥6...
3,4.0,Pasien menderita demam disertai munculnya brun...,"Infeksi Bakteri/Virus (Sistemik) 50%, Infeksi ...","DBD, Infeksi Bakteri/Virus (Sistemik), Infeksi...","Demam Biasa, Dispepsia, Asma, Infeksi Lokal, A...","Infeksi Kulit (Dermatitis atau Selulitis), Kej...","Infeksi Bakteri (Selulitis), Reaksi Alergi (De...",Erisipelas (Infeksi Bakteri pada Kulit dan Jar...,"Berdasarkan informasi klinis yang diberikan, s...","Berdasarkan gejala yang disampaikan, yaitu dem...","Berdasarkan informasi yang diberikan, pasien m...",### Analisis Gejala dan Kemungkinan Diagnosis\...,Berdasarkan gejala dan temuan pemeriksaan yang...
4,5.0,Pasien nyeri dada sejak 4 jam lalu seperti ter...,"Angina Pekrotis 80%, Infark Miokard Akut (Sera...","Angina Pekrotis, Infark Miokard Akut (Serangan...","Infark Miokard Akut (Serangan Jantung), Dispep...","Angina Pekrotis, Infark Miokard, Penyakit Musk...","Angina Pectoris (Kardiovaskular), Infark Mioka...","Infark Miokard Akut (Serangan Jantung), Angina...",Berdasarkan informasi yang diberikan dalam kon...,"Berdasarkan gejala yang disampaikan, yaitu nye...","Berdasarkan informasi yang diberikan, pasien m...",### Analisis Gejala\n\nPasien mengeluhkan nyer...,Berdasarkan gejala nyeri dada seperti terhimpi...
5,6.0,Pasien mengalami sesak nafas sejak siang. #Has...,"Asma 100%, Bronkitis Akut 70%, Penyakit Paru O...","Asma, Bronkitis Akut, Pnemonia, Penyakit Paru ...","Asma, Pneumonia, Dispepsia, Demam biasa, Kardi...","Asma, Penyakit Paru Obstruktif Kronis, Pnemoni...","Asma, Bronkitis Akut, Pneumotoraks, Gagal Jan...","Asma Akut Eksaserbasi, PPOK Eksaserbasi Akut...",Berdasarkan informasi klinis yang diberikan da...,"Berdasarkan gejala yang disampaikan, yaitu ses...","Berdasarkan informasi yang diberikan, pasien m...",Berdasarkan gejala dan hasil pemeriksaan yang ...,Berdasarkan gejala sesak napas disertai wheezi...
6,7.0,Pasien mengalami sesak napas sejak sehari lalu...,Bronkitis Akut 100%,"Pneumonia Komunitas, Bronk

In [ ]:
comparison_df.to_csv('ready_to_eval.csv')